# ONNX Model Converter 

## 1. Cấu Hình Model

Thay đổi các giá trị bên dưới để cấu hình model cần chuyển đổi.

In [ ]:
# ============================================================
# CẤU HÌNH - THAY ĐỔI CÁC GIÁ TRỊ NÀY
# ============================================================

# Model HuggingFace cần chuyển đổi
MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"

# Task của model (auto = tự động detect, hoặc chọn thủ công)
# Các giá trị: "auto", "feature-extraction", "text-classification", "text-generation"
TASK = "auto"

# Tên output (để trống = tự động tạo từ MODEL_ID)
OUTPUT_NAME = ""

# HuggingFace username của bạn
HF_USERNAME = "n24q02m"

# Bật/tắt quantization INT8
ENABLE_QUANTIZATION = True

# ============================================================
# KHÔNG CẦN THAY ĐỔI PHẦN DƯỚI
# ============================================================

## 2. Thiết Lập Môi Trường

Tự động phát hiện và cấu hình môi trường (local/colab/kaggle)

In [ ]:
import sys
import os
import gc
import subprocess
import shutil
import time
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")


# Phát hiện môi trường
def detect_env():
    if "google.colab" in sys.modules:
        return "colab"
    elif "kaggle_web_client" in sys.modules or os.path.exists("/kaggle"):
        return "kaggle"
    else:
        return "local"


ENV = detect_env()
print(f"Môi trường: {ENV.upper()}")

# Thiết lập thư mục output
if ENV == "colab":
    OUTPUT_DIR = Path("/content/models")
elif ENV == "kaggle":
    OUTPUT_DIR = Path("/kaggle/working/models")
else:
    OUTPUT_DIR = Path.cwd() / "models"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Thư mục output: {OUTPUT_DIR}")

## 3. Import Thư Viện

In [ ]:
import torch
import onnx
import onnxruntime as ort
from transformers import AutoTokenizer, AutoConfig
from optimum.onnxruntime import (
    ORTModelForFeatureExtraction,
    ORTModelForSequenceClassification,
)


def clear_memory():
    """Giải phóng bộ nhớ RAM và GPU"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


print(f"PyTorch: {torch.__version__}")
print(f"ONNX: {onnx.__version__}")
print(f"ONNX Runtime: {ort.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

## 4. Xử Lý Cấu Hình

In [ ]:
# Tự động detect task từ model_id
def detect_task(model_id):
    """Tự động detect task dựa trên tên model"""
    model_lower = model_id.lower()
    if "embedding" in model_lower:
        return "feature-extraction"
    elif "reranker" in model_lower or "ranker" in model_lower:
        return "text-classification"
    elif "classifier" in model_lower or "classification" in model_lower:
        return "text-classification"
    else:
        # Thử đọc config để detect
        try:
            config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
            if hasattr(config, "num_labels") and config.num_labels > 0:
                return "text-classification"
        except:
            pass
        return "feature-extraction"


# Xử lý cấu hình
if TASK == "auto":
    TASK = detect_task(MODEL_ID)
    print(f"Task tự động detect: {TASK}")

if not OUTPUT_NAME:
    # Tạo tên từ MODEL_ID
    OUTPUT_NAME = MODEL_ID.split("/")[-1] + "-ONNX"

# Tạo thư mục cho model
MODEL_OUTPUT_DIR = OUTPUT_DIR / OUTPUT_NAME.lower().replace(" ", "-")
FP32_DIR = MODEL_OUTPUT_DIR / "fp32"
INT8_DIR = MODEL_OUTPUT_DIR / "int8"

MODEL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FP32_DIR.mkdir(parents=True, exist_ok=True)
if ENABLE_QUANTIZATION:
    INT8_DIR.mkdir(parents=True, exist_ok=True)

# Chọn ORT class phù hợp
if TASK == "feature-extraction":
    ORTModelClass = ORTModelForFeatureExtraction
elif TASK == "text-classification":
    ORTModelClass = ORTModelForSequenceClassification
else:
    raise ValueError(f"Task không được hỗ trợ: {TASK}")

print("\n" + "=" * 50)
print("CẤU HÌNH CUỐI CÙNG")
print("=" * 50)
print(f"Model ID: {MODEL_ID}")
print(f"Task: {TASK}")
print(f"Output Name: {OUTPUT_NAME}")
print(f"Output Dir: {MODEL_OUTPUT_DIR}")
print(f"Quantization: {ENABLE_QUANTIZATION}")
print(f"HF Username: {HF_USERNAME}")

## 5. Export Model sang ONNX (FP32)

In [ ]:
print("=" * 50)
print("EXPORT ONNX (FP32)")
print("=" * 50)

print(f"\nĐang tải và export model: {MODEL_ID}")
print("Quá trình này có thể mất vài phút...")

export_success = False

try:
    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

    # Export sang ONNX
    ort_model = ORTModelClass.from_pretrained(
        MODEL_ID, export=True, trust_remote_code=True
    )

    # Lưu model và tokenizer
    ort_model.save_pretrained(FP32_DIR)
    tokenizer.save_pretrained(FP32_DIR)

    print(f"\nModel đã được export tại: {FP32_DIR}")

    # Hiển thị files
    print("\nDanh sách files:")
    total_size = 0
    for f in FP32_DIR.iterdir():
        if f.is_file():
            size_mb = f.stat().st_size / (1024**2)
            total_size += size_mb
            print(f"  - {f.name} ({size_mb:.2f} MB)")
    print(f"  Tổng: {total_size:.2f} MB")

    # Giải phóng bộ nhớ
    del ort_model, tokenizer
    clear_memory()
    export_success = True

except Exception as e:
    print(f"\nLỗi khi export: {e}")
    print("\nThử phương pháp CLI...")

    try:
        cmd = f'optimum-cli export onnx --model "{MODEL_ID}" --task {TASK} --trust-remote-code "{FP32_DIR}"'
        result = subprocess.run(
            cmd, shell=True, capture_output=True, text=True, timeout=600
        )

        if list(FP32_DIR.glob("*.onnx")):
            print("Export thành công qua CLI!")
            export_success = True
        else:
            print(f"CLI thất bại: {result.stderr}")
    except Exception as e2:
        print(f"CLI cũng thất bại: {e2}")

    clear_memory()

if export_success:
    print("\nExport FP32: THÀNH CÔNG")
else:
    print("\nExport FP32: THẤT BẠI")
    raise Exception("Không thể export model sang ONNX")

## 6. Quantization (INT8)

Áp dụng Dynamic Quantization để giảm kích thước model.

In [ ]:
quantize_success = False

if ENABLE_QUANTIZATION:
    print("=" * 50)
    print("QUANTIZATION (INT8)")
    print("=" * 50)

    print("\nĐang quantize model...")
    print("(Có thể mất vài phút)")

    # Phương pháp: Sử dụng subprocess.Popen để tránh timeout
    cmd = f'optimum-cli onnxruntime quantize --onnx_model "{FP32_DIR}" --avx512_vnni -o "{INT8_DIR}"'

    try:
        # Chạy với Popen để có thể monitor
        process = subprocess.Popen(
            cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
        )

        # Đọc output real-time
        output_lines = []
        for line in process.stdout:
            print(line, end="")
            output_lines.append(line)

        process.wait(timeout=600)  # 10 phút timeout

        # Đợi file được ghi xong
        time.sleep(3)

    except subprocess.TimeoutExpired:
        process.kill()
        print("\nTimeout - process đã bị kill")
    except Exception as e:
        print(f"\nLỗi: {e}")

    # Kiểm tra kết quả
    int8_files = list(INT8_DIR.glob("*.onnx"))

    if int8_files:
        print("\nQuantization: THÀNH CÔNG")
        quantize_success = True

        # Copy tokenizer files
        for f in FP32_DIR.iterdir():
            if not f.name.endswith(".onnx") and not f.name.endswith("_data"):
                if f.is_file() and not (INT8_DIR / f.name).exists():
                    shutil.copy(f, INT8_DIR / f.name)

        # So sánh kích thước
        fp32_size = sum(f.stat().st_size for f in FP32_DIR.glob("*.onnx")) / (1024**2)
        int8_size = sum(f.stat().st_size for f in INT8_DIR.glob("*.onnx")) / (1024**2)

        print(f"\nSo sánh kích thước:")
        print(f"  FP32: {fp32_size:.2f} MB")
        print(f"  INT8: {int8_size:.2f} MB")
        if fp32_size > 0:
            print(f"  Giảm: {(1 - int8_size / fp32_size) * 100:.1f}%")
    else:
        print("\nQuantization: THẤT BẠI")
        print("Chỉ có model FP32 khả dụng.")

    clear_memory()
else:
    print("Quantization đã tắt. Bỏ qua...")

## 7. Kiểm Tra Model

In [ ]:
print("=" * 50)
print("KIỂM TRA MODEL")
print("=" * 50)


def test_model(model_path, model_name, task):
    """Test model ONNX"""
    try:
        if not list(Path(model_path).glob("*.onnx")):
            print(f"{model_name}: Không tìm thấy file ONNX")
            return False

        tokenizer = AutoTokenizer.from_pretrained(model_path)

        if task == "feature-extraction":
            model = ORTModelForFeatureExtraction.from_pretrained(model_path)
            test_text = "This is a test sentence."
            inputs = tokenizer(
                test_text, return_tensors="pt", padding=True, truncation=True
            )

            # Thêm position_ids nếu cần (cho Qwen models)
            if "position_ids" not in inputs:
                inputs["position_ids"] = torch.arange(
                    inputs["input_ids"].shape[1]
                ).unsqueeze(0)

            outputs = model(**inputs)
            print(f"{model_name}: OK (output shape: {outputs.last_hidden_state.shape})")

        elif task == "text-classification":
            model = ORTModelForSequenceClassification.from_pretrained(model_path)
            query = "What is machine learning?"
            doc = "Machine learning is a branch of AI."
            inputs = tokenizer(
                query, doc, return_tensors="pt", padding=True, truncation=True
            )
            outputs = model(**inputs)
            print(f"{model_name}: OK (logits: {outputs.logits.shape})")

        del model, tokenizer
        clear_memory()
        return True

    except Exception as e:
        print(f"{model_name}: LỖI - {e}")
        clear_memory()
        return False


# Test FP32
print("\n--- Test FP32 ---")
fp32_ok = test_model(FP32_DIR, "FP32", TASK)

# Test INT8
if ENABLE_QUANTIZATION and quantize_success:
    print("\n--- Test INT8 ---")
    int8_ok = test_model(INT8_DIR, "INT8", TASK)
else:
    int8_ok = False

## 8. Tạo Model Card

In [ ]:
print("=" * 50)
print("TẠO MODEL CARD")
print("=" * 50)

# Xác định model type
model_type = "embedding" if TASK == "feature-extraction" else "reranker"
ort_class = (
    "FeatureExtraction" if TASK == "feature-extraction" else "SequenceClassification"
)

# Xác định versions
versions = ["FP32"]
if ENABLE_QUANTIZATION and quantize_success:
    versions.append("INT8 Dynamic")

model_card = f'''---
license: apache-2.0
tags:
  - onnx
  - optimum
  - {model_type}
base_model: {MODEL_ID}
library_name: optimum
pipeline_tag: {TASK}
---

# {OUTPUT_NAME}

ONNX version of [{MODEL_ID}](https://huggingface.co/{MODEL_ID})

## Model Information

| Property | Value |
|----------|-------|
| Source Model | [{MODEL_ID}](https://huggingface.co/{MODEL_ID}) |
| Format | ONNX |
| Versions | {" + ".join(versions)} |
| Task | {TASK} |

## Available Versions

- `fp32/`: Full precision (FP32)
{"- `int8/`: Quantized INT8 (dynamic quantization)" if ENABLE_QUANTIZATION and quantize_success else ""}

## Usage

### With Optimum

```python
from optimum.onnxruntime import ORTModelFor{ort_class}
from transformers import AutoTokenizer

model = ORTModelFor{ort_class}.from_pretrained(
    "{HF_USERNAME}/{OUTPUT_NAME}",
    subfolder="fp32"
)
tokenizer = AutoTokenizer.from_pretrained(
    "{HF_USERNAME}/{OUTPUT_NAME}",
    subfolder="fp32"
)
```

## License

Apache 2.0 (following the original model license)

## Credits

- Original model: [{MODEL_ID}](https://huggingface.co/{MODEL_ID})
- ONNX conversion: [Optimum](https://huggingface.co/docs/optimum)
'''

# Lưu Model Card
readme_path = MODEL_OUTPUT_DIR / "README.md"
readme_path.write_text(model_card, encoding="utf-8")
print(f"Đã tạo: {readme_path}")

## 9. Hướng Dẫn Upload

In [ ]:
print("=" * 50)
print("HƯỚNG DẪN UPLOAD")
print("=" * 50)

print(f"""
=== CÁCH 1: HuggingFace CLI ===

# Đăng nhập
huggingface-cli login

# Tạo repo
huggingface-cli repo create {OUTPUT_NAME} --type model

# Upload
huggingface-cli upload {HF_USERNAME}/{OUTPUT_NAME} "{MODEL_OUTPUT_DIR}" .

=== CÁCH 2: Web UI ===

1. Truy cập: https://huggingface.co/new
2. Tạo repo: {OUTPUT_NAME}
3. Upload files từ: {MODEL_OUTPUT_DIR}

=== CẤU TRÚC FILES ===
""")

# Hiển thị cấu trúc
for item in sorted(MODEL_OUTPUT_DIR.rglob("*")):
    rel_path = item.relative_to(MODEL_OUTPUT_DIR)
    indent = "  " * (len(rel_path.parts) - 1)
    if item.is_file():
        size_mb = item.stat().st_size / (1024**2)
        print(f"{indent}{item.name} ({size_mb:.2f} MB)")
    else:
        print(f"{indent}{item.name}/")

## 10. Tổng Kết

In [ ]:
print("\n" + "=" * 50)
print("TỔNG KẾT")
print("=" * 50)

print(f"\nModel: {MODEL_ID}")
print(f"Task: {TASK}")
print(f"Output: {MODEL_OUTPUT_DIR}")


# Kích thước
def get_folder_size(folder):
    return sum(f.stat().st_size for f in Path(folder).rglob("*") if f.is_file()) / (
        1024**2
    )


print(f"\nKích thước:")
try:
    total_size = get_folder_size(MODEL_OUTPUT_DIR)
    print(f"  Tổng: {total_size:.2f} MB")
except:
    pass

print(f"\nKết quả:")
print(f"  FP32: {'OK' if fp32_ok else 'LỖI'}")
if ENABLE_QUANTIZATION:
    print(f"  INT8: {'OK' if quantize_success and int8_ok else 'KHÔNG KHẢ DỤNG'}")

print(f"\nURL sau khi upload:")
print(f"  https://huggingface.co/{HF_USERNAME}/{OUTPUT_NAME}")

print("\n" + "=" * 50)
print("HOÀN THÀNH!")
print("=" * 50)